In [1]:
from datetime import datetime

import numpy as np
import talib as ta

from btopt.engine import Engine
from btopt.log_config import logger_test
from btopt.order import Order
from btopt.strategy.strategy import Strategy
from btopt.util.ext_decimal import ExtendedDecimal

In [2]:
def parse_date(epoch):
    return datetime.fromtimestamp(epoch / 1_000_000_000).strftime("%Y-%m-%d %H:%M:%S")


class SimpleMovingAverageCrossover(Strategy):
    def __init__(self, fast_period: int = 10, slow_period: int = 20, risk_percent=0.1):
        self.fast_period = fast_period
        self.slow_period = slow_period
        self.risk_percent = risk_percent
        self.warmup_period = 1

        self.fast_ma = None
        self.slow_ma = None

        self.total = 0

    def on_data(self) -> None:
        bar = self.datas[self._primary_symbol].get(index=0)
        close_prices = self.datas[self._primary_symbol].get(
            size=self.slow_period, value="close"
        )[::-1]

        if len(close_prices) < self.slow_period:
            return

        fast_ma = ta.SMA(
            np.array(close_prices).astype(float), timeperiod=self.fast_period
        )[-1]
        slow_ma = ta.SMA(
            np.array(close_prices).astype(float), timeperiod=self.slow_period
        )[-1]

        current_position = self.get_current_position(bar.ticker)
        if (self.fast_ma is not None) and (self.slow_ma is not None):
            prev_diff = self.fast_ma - self.slow_ma
            diff = fast_ma - slow_ma

            if (diff > 0) and (prev_diff <= 0):
                logger_test.warning(f"New Long: Current Position: {current_position}")
                # Bullish crossover
                if current_position <= 0:
                    position_size = self.calculate_position_size(
                        bar.ticker,
                        bar.close,
                        self.risk_percent,
                    )

                    size = abs(current_position) + position_size
                    self.buy(bar.ticker, size)
                    self.total += 1
            elif (diff < 0) and (prev_diff >= 0):
                logger_test.warning(f"New Short: Current Position: {current_position}")
                # Bearish crossover
                if current_position >= 0:
                    position_size = self.calculate_position_size(
                        bar.ticker,
                        bar.close,
                        self.risk_percent,
                    )

                    size = abs(current_position) + position_size
                    self.sell(bar.ticker, size)
                    self.total += 1

        # Update MA values for next iteration
        self.fast_ma = fast_ma
        self.slow_ma = slow_ma

    def on_order_update(self, order: Order) -> None:
        # if order.status == order.Status.FILLED:
        #     logger_test.info(
        #         f"Order {order.id} filled | Time = {order.get_last_fill_timestamp()}; Price = {order.get_last_fill_price()}; Status: {order.status}"
        #     )
        ...

    def on_trade_update(self, trade) -> None:
        # if trade.status == trade.Status.CLOSED:
        #     logger_test.info(
        #         f"Trade {trade.id} Closed | Time = {trade.exit_timestamp}; Exit Price = {trade.exit_price}; PnL = {trade.metrics.pnl}"
        #     )
        ...

In [3]:
from btopt.data.dataloader import CSVDataLoader


def run_backtest():
    # Initialize the engine
    engine = Engine()

    # Load data

    start_date = "2021-01-01"
    end_date = "2023-01-01"

    symbol = "EURUSD"
    dataloader = CSVDataLoader(symbol, "1m", start_date=start_date, end_date=end_date)
    engine.resample_data(dataloader, "1d")

    # symbol = "ES=F"
    # dataloader = YFDataloader(symbol, "1d", start_date=start_date, end_date=end_date)
    # engine.add_data(dataloader)

    # # Create and add the strategy
    # strategy = SimpleMovingAverageCrossover(
    #     "SMA Crossover", fast_period=10, slow_period=20
    # )

    # engine.add_strategy(Empty, ctf, htf)
    engine.add_strategy(SimpleMovingAverageCrossover, fast_period=10, slow_period=20)

    # Set up the backtest configuration

    initial_capital = ExtendedDecimal("100000")
    commission_rate = ExtendedDecimal("0.000")  # 0.1% commission
    config = {
        "initial_capital": initial_capital,
        "commission_rate": commission_rate,
    }
    engine.set_config(config)

    # Run the backtest
    try:
        logger_test.info("Starting backtest")
        reporter = engine.run()

        return reporter
    except Exception as e:
        logger_test.error(f"Error during backtest: {e}", exc_info=True)

In [6]:
reporter = run_backtest()

/Users/jerryinyang/Code/btopt/btopt/portfolio.py:217: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.metrics = pd.concat([self.metrics, new_row], ignore_index=True)
ERROR [main]
Error during backtest: Portfolio._update_margin_and_buying_power() missing 2 required positional arguments: 'order' and 'cost'
  File "/Users/jerryinyang/Code/btopt/btopt/engine.py", line 491
     Source: /Users/jerryinyang/Code/btopt/btopt/util/logger.py:141
Traceback (most recent call last):
  File "/Users/jerryinyang/Code/btopt/btopt/engine.py", line 481, in run
    self._process_timestamp(timestamp, self._current_market_data)
  File "/Users/jerryinyang/Code/btopt/btopt/engine.py", line 559, in _process_timestamp
    self.portfolio.update(timestamp, data_poin

In [7]:
reporter.plot_equity_curve()

AttributeError: 'NoneType' object has no attribute 'plot_equity_curve'